In [1]:
# learn how to enlarge the block in dmrg
# from one site enlarge to two site, 系统从一个各点扩充到两个各点。
import numpy as np
from scipy.sparse import kron, identity
from collections import namedtuple
def H2(Sz1, Sp1, Sz2, Sp2):  # two-site part of H
    """Given the operators S^z and S^+ on two sites in different Hilbert spaces
    (e.g. two blocks), returns a Kronecker product representing the
    corresponding two-site term in the Hamiltonian that joins the two sites.
    """
    J = Jz = 1.
    return (
        (J / 2) * (kron(Sp1, Sp2.conjugate().transpose()) + kron(Sp1.conjugate().transpose(), Sp2)) +
        Jz * kron(Sz1, Sz2)
    )
if __name__ == "__main__":
    # Model-specific code for the Heisenberg XXZ chain
    model_d = 2  # single-site basis size
    Sz1 = np.array([[0.5, 0], [0, -0.5]], dtype='d')  # single-site S^z
    Sp1 = np.array([[0, 1], [0, 0]], dtype='d')  # single-site S^+
    H1 = np.array([[0, 0], [0, 0]], dtype='d')  # single-site portion of H is zero, 磁场项 或者势能项
    Block = namedtuple("Block", ["length", "basis_size", "operator_dict"])
    EnlargedBlock = namedtuple("EnlargedBlock", ["length", "basis_size", "operator_dict"])
    #初始化一个block
    initial_block = Block(length=1, 
                        basis_size=model_d, 
                        operator_dict={
                        "H": H1,
                        "conn_Sz": Sz1,
                        "conn_Sp": Sp1,})
    block = initial_block
    # enlarges the provided Block by a single site, return an EnlargedBlock
    mblock = block.basis_size
    o = block.operator_dict

    H = kron(o["H"], identity(model_d)) + kron(identity(mblock),H1) + \
        H2(o["conn_Sz"], o["conn_Sp"], Sz1, Sp1) # hamiltonian of block
    conn_Sz = kron(identity(mblock), Sz1) # edge site is enlarged
    conn_Sp = kron(identity(mblock), Sp1) # edge site is enlarged

    enlarged_operator_dict = {"H": H, "conn_Sz": conn_Sz, "conn_Sp": conn_Sp}
    EnlargedBlock(length=(block.length + 1),
                basis_size=(block.basis_size * model_d),
                operator_dict=enlarged_operator_dict) # 存入 EnlareBlock 中。

* 得到 enlarged block hamiltonian H 的时候， H 中包含三项：
（1） kron(o["H"], identity(model_d)) ： 将原 block hamiltonian H 的希尔伯特空间扩大一个格点
（2） kron(identity(mblock),H1)： 将新格点的势能项添加进来
（3） H2(o["conn_Sz"], o["conn_Sp"], Sz1, Sp1)： 将原 block 边缘格点与新加入的格点之间的相互作用添加进来

In [2]:
print("block hamiltonian H:\n", H)
print("edge site operator conn_Sz:\n", conn_Sz)
print("edge site operator conn_Sp:\n", conn_Sp)

block hamiltonian H:
   (0, 0)	0.25
  (1, 1)	-0.25
  (1, 2)	0.5
  (2, 1)	0.5
  (2, 2)	-0.25
  (3, 3)	0.25
edge site operator conn_Sz:
   (0, 0)	0.5
  (0, 1)	0.0
  (1, 0)	0.0
  (1, 1)	-0.5
  (2, 2)	0.5
  (2, 3)	0.0
  (3, 2)	0.0
  (3, 3)	-0.5
edge site operator conn_Sp:
   (0, 1)	1.0
  (2, 3)	1.0
